# Video Highlights Generation

This notebook generates video highlights from a video file using Azure AI services.

## Pre-requisites
1. Install required packages

In [1]:
import subprocess
import sys
import os

# Install only essential packages needed for the pipeline
# Using a minimal, reliable set to avoid dependency conflicts
essential_packages = [
    'openai',
    'python-dotenv',
    'requests',
    'jsonschema',
    'moviepy',
    'Pillow',
    'azure-identity'
]

print("Installing essential packages...")
failed_packages = []

for package in essential_packages:
    try:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package], 
                            capture_output=False, text=True)
        print(f"✓ Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to install {package}")
        failed_packages.append(package)

if failed_packages:
    print(f"\nFailed packages: {failed_packages}")
    print("The notebook may still work if these are optional dependencies.")
else:
    print("\n✓ All essential packages installed successfully!")

print("\nPackage installation complete!")

# Test critical imports
print("\nTesting critical imports...")
try:
    import moviepy.editor
    print("✓ moviepy import successful")
except ImportError as e:
    print(f"✗ moviepy import failed: {e}")

try:
    import openai
    print("✓ openai import successful")
except ImportError as e:
    print(f"✗ openai import failed: {e}")

try:
    import jsonschema
    print("✓ jsonschema import successful")
except ImportError as e:
    print(f"✗ jsonschema import failed: {e}")

print("\nReady to proceed with the notebook!")

Installing essential packages...
Installing openai...


TypeError: Popen.__init__() got an unexpected keyword argument 'capture_output'

## Load environment variables

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)

# Add necessary environment variables here
# e.g., AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

False

## Pipeline Configuration

Configure the settings for the highlight generation pipeline.
**Important:** You must specify the `SOURCE_VIDEO_PATH` before running the notebook.

In [3]:
import os
import time

# --- REQUIRED: SPECIFY YOUR VIDEO PATH ---
SOURCE_VIDEO_PATH = r"C:\Users\t-kjindel\OneDrive - Microsoft\Demo Samples\Satya Build Keynote (Density - 2.5, Time - 100s).mp4" # e.g., "path/to/your/video.mp4"

# --- Optional: Adjust other settings ---
OUTPUT_DIR = os.path.dirname(os.path.abspath(SOURCE_VIDEO_PATH if SOURCE_VIDEO_PATH else __file__))
VIDEO_TYPE = "eric and asha keynote ai event"  # See available schemas in the 'schemas' folder
CLIP_DENSITY = "medium"
TARGET_DURATION_S = 120
PERSONALIZATION = "exciting moments"
HUMAN_IN_THE_LOOP_REVIEW = True  # Set to True to enable human review
ADD_CAPTIONS = True
TRANSITION_TYPE = "fade"
RESOLUTION = 720
SPEED_RAMP = False

# --- File Paths ---
# These will be populated as the pipeline runs
SCHEMA_PATH = ""
ANALYSIS_RESULT_PATH = ""
PREFILTERED_SEGMENTS_PATH = ""
HIGHLIGHT_PLAN_PATH = ""
OUTPUT_HIGHLIGHT_PATH = os.path.join(OUTPUT_DIR, "highlight.mp4")
EVALUATION_REPORT_PATH = ""

# --- Analyzer ID ---
ANALYZER_ID = f"auto-highlight-analyzer-{int(time.time())}"

# Basic validation
if not SOURCE_VIDEO_PATH:
    raise ValueError("SOURCE_VIDEO_PATH cannot be empty. Please specify the path to your video file.")

print(f"Source Video: {SOURCE_VIDEO_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")

Source Video: C:\Users\t-kjindel\OneDrive - Microsoft\Demo Samples\Satya Build Keynote (Density - 2.5, Time - 100s).mp4
Output Directory: C:\Users\t-kjindel\OneDrive - Microsoft\Demo Samples


## 1. Generate and Activate Schema

First, we need to generate or activate a schema that tells the analysis service what to look for in the video. The schema is based on the `VIDEO_TYPE`, `CLIP_DENSITY`, `TARGET_DURATION_S`, and `PERSONALIZATION` settings defined above.

In [4]:
import schema_manager

try:
    SCHEMA_PATH = schema_manager.activate_schema(
        VIDEO_TYPE, 
        CLIP_DENSITY,
        TARGET_DURATION_S,
        PERSONALIZATION,
        human_in_the_loop_review=HUMAN_IN_THE_LOOP_REVIEW
    )
    print(f"Schema activated: {SCHEMA_PATH}")
    
    # Display schema content
    with open(SCHEMA_PATH, 'r') as f:
        import json
        print(json.dumps(json.load(f), indent=2))

except Exception as e:
    print(f"Error activating schema: {e}")


Activated schema for 'eric and asha keynote ai event' -> c:\Users\t-kjindel\OneDrive - Microsoft\Desktop\Highlights Generation Notebook\video_analysis_schema.json
Schema preview (first 10 lines):
{
  "description": "Eric and Asha Keynote AI Event highlight analyzer \u2013 segments video at key narrative moments and extracts flat metadata for each clip.",
  "baseAnalyzerId": "prebuilt-videoAnalyzer",
  "config": {
    "returnDetails": true,
    "segmentationMode": "custom",
    "segmentationDefinition": "Segment the video at important or exciting moments of the Eric and Asha Keynote AI Event. Focus on new announcements, demos, Q&A interactions, and other high-engagement segments. Target a medium clip density, with an overall highlight reel totaling around 120 seconds. Prioritize personalization by surfacing the most exciting moments."
  },
  "fieldSchema": {
    "fields": {

Schema activated: c:\Users\t-kjindel\OneDrive - Microsoft\Desktop\Highlights Generation Notebook\video_analysis_

## 2. Analyze Video

Now, we submit the video for analysis using the generated schema. This step can take a long time depending on the length of the video.

In [5]:
import run_video_analysis
import json

try:
    # This script runs the video analysis using the run_analysis function
    success, ANALYSIS_RESULT_PATH, error = run_video_analysis.run_analysis(
        video_path=SOURCE_VIDEO_PATH,
        schema_path=SCHEMA_PATH,
        analyzer_id=ANALYZER_ID
    )
    
    if success:
        print(f"Video analysis complete. Results saved to: {ANALYSIS_RESULT_PATH}")
        
        # Display a snippet of the analysis result
        with open(ANALYSIS_RESULT_PATH, 'r') as f:
            result_data = json.load(f)
            print(json.dumps(result_data, indent=2, sort_keys=True)[:1000] + "\n...")
    else:
        print(f"Video analysis failed: {error}")

except Exception as e:
    print(f"Error during video analysis: {e}")

--- Starting Video Analysis: Satya Build Keynote (Density - 2.5, Time - 100s).mp4 ---
Using analyzer ID: auto-highlight-analyzer-1755020560
Attempting to authenticate using Azure CLI credentials...
Successfully authenticated using Azure CLI.

Attempting to create/update analyzer 'soccer-highlights-analyzer5314167881755020564'...
Successfully authenticated using Azure CLI.

Attempting to create/update analyzer 'soccer-highlights-analyzer5314167881755020564'...
Successfully created or updated analyzer 'soccer-highlights-analyzer5314167881755020564'.

Starting analysis for local video file: 'C:\Users\t-kjindel\OneDrive - Microsoft\Demo Samples\Satya Build Keynote (Density - 2.5, Time - 100s).mp4'...
Successfully created or updated analyzer 'soccer-highlights-analyzer5314167881755020564'.

Starting analysis for local video file: 'C:\Users\t-kjindel\OneDrive - Microsoft\Demo Samples\Satya Build Keynote (Density - 2.5, Time - 100s).mp4'...
Analysis job started. You can check the results at: 

## 3. Parse and Pre-filter Segments

Next, we parse the raw analysis output and pre-filter the segments based on the schema's scoring. This creates a simplified list of potential highlight clips.

In [10]:
import json_parser
import json

try:
    PREFILTERED_SEGMENTS_PATH, _, _ = json_parser.process_json(
        input_path=ANALYSIS_RESULT_PATH
    )
    print(f"Segments parsed and pre-filtered. Results saved to: {PREFILTERED_SEGMENTS_PATH}")

    # Display the pre-filtered segments
    with open(PREFILTERED_SEGMENTS_PATH, 'r') as f:
        segments_data = json.load(f)
        print(json.dumps(segments_data, indent=2))

except Exception as e:
    print(f"Error parsing segments: {e}")

Parsed 1 segments -> kept 1
Kept 100.0% of segments
Segments parsed and pre-filtered. Results saved to: C:\Users\t-kjindel\OneDrive - Microsoft\Demo Samples\prefiltered_segments_analysis_result_Satya Build Keynote (Density - 2.5, Time - 100s).json
[
  {
    "SegmentId": "1",
    "SegmentType": "KeynoteIntro",
    "Speaker": "Unknown",
    "AudienceReaction": "Medium",
    "AnnouncementType": "None",
    "QandAQuestions": "",
    "HighlightScore": 0.75,
    "StartTimeMs": 891,
    "EndTimeMs": 99495,
    "Description": "The event begins with a welcome message to Microsoft Build. The stage is set with the Microsoft logo prominently displayed. The speaker enters the stage and begins the keynote, introducing the event and setting the context. The focus shifts to GitHub, indicating a discussion or announcement related to it. The speaker continues to discuss GitHub as the world's open developer platform. The segment continues with a demonstration of GitHub Copilot, showcasing its capabilitie

## 4. Build Highlight Plan

Using the pre-filtered segments, we now build the final highlight plan. This involves selecting the best clips to meet the `TARGET_DURATION_S` and arranging them in a compelling order.

In [13]:
import build_highlight
import json

try:
    # Set the global variables that build_highlight.main() expects
    build_highlight.SEGMENTS_PATH = PREFILTERED_SEGMENTS_PATH
    build_highlight.RUNTIME_S = TARGET_DURATION_S
    
    # Call main function (no parameters needed)
    build_highlight.main()
    
    # The output file is hardcoded to "final_highlight_result.json" in the same directory
    HIGHLIGHT_PLAN_PATH = "final_highlight_result.json"
    print(f"Highlight plan created. Saved to: {HIGHLIGHT_PLAN_PATH}")

    # Display the highlight plan
    with open(HIGHLIGHT_PLAN_PATH, 'r') as f:
        plan_data = json.load(f)
        print(json.dumps(plan_data, indent=2))

except Exception as e:
    print(f"Error building highlight plan: {e}")

=== USER MESSAGE SENT TO MODEL ===
{
  "video_type": "table tennis",
  "clip_density": 1.0,
  "target_duration_s": 120,
  "personalization": "none",
  "Segments": [
    {
      "SegmentId": "1",
      "SegmentType": "KeynoteIntro",
      "Speaker": "Unknown",
      "AudienceReaction": "Medium",
      "AnnouncementType": "None",
      "QandAQuestions": "",
      "HighlightScore": 0.75,
      "StartTimeMs": 891,
      "EndTimeMs": 99495,
      "Description": "The event begins with a welcome message to Microsoft Build. The stage is set with the Microsoft logo prominently displayed. The speaker enters the stage and begins the keynote, introducing the event and setting the context. The focus shifts to GitHub, indicating a discussion or announcement related to it. The speaker continues to discuss GitHub as the world's open developer platform. The segment continues with a demonstration of GitHub Copilot, showcasing its capabilities and integration. The speaker emphasizes the importance of bui

## 5. Stitch Video Clips

With the highlight plan in place, we can now stitch the selected video clips together to create the final highlight video. This step uses `moviepy` to perform the video editing.

In [16]:
import video_stitching

try:
    video_stitching.main(
        source_video_path=SOURCE_VIDEO_PATH,
        highlight_plan_path=HIGHLIGHT_PLAN_PATH,
        output_path=OUTPUT_HIGHLIGHT_PATH,
        add_captions=ADD_CAPTIONS,
        transition=TRANSITION_TYPE,
        resolution=RESOLUTION,
        speed_ramp=SPEED_RAMP
    )
    print(f"Highlight video successfully created at: {OUTPUT_HIGHLIGHT_PATH}")

except Exception as e:
    print(f"Error stitching video: {e}")

ModuleNotFoundError: No module named 'moviepy.editor'